## Data Quality Dashboard in Python

**Description**: Create a basic dashboard using a Python library (e.g., Plotly Dash) to visualize data quality metrics for a given dataset.

In [2]:
!pip install dash pandas plotly

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Write your code from here

import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Sample dataset (can replace with your own CSV)
df = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', None, 'Eve'],
    'Age': [25, None, 30, 22, 29],
    'Email': ['a@example.com', 'b@example.com', None, 'd@example.com', 'e@example.com'],
    'Salary': [50000, 60000, None, 55000, 52000]
})

# Calculate quality metrics
def calculate_quality_metrics(data):
    return pd.DataFrame({
        'Column': data.columns,
        'Missing Values': data.isnull().sum().values,
        'Missing (%)': (data.isnull().mean() * 100).round(2),
        'Unique Values': data.nunique().values,
        'Data Type': data.dtypes.astype(str).values
    })

# Initialize app
app = dash.Dash(__name__)
app.title = 'Data Quality Dashboard'

# Layout
app.layout = html.Div([
    html.H1("📊 Data Quality Dashboard", style={'textAlign': 'center'}),

    html.Div(id='summary-table'),

    html.Br(),
    dcc.Graph(id='missing-bar'),

    html.Br(),
    dcc.Graph(id='unique-bar'),
])

# Callbacks
@app.callback(
    Output('summary-table', 'children'),
    Output('missing-bar', 'figure'),
    Output('unique-bar', 'figure'),
    Input('summary-table', 'children')
)
def update_dashboard(_):
    quality_df = calculate_quality_metrics(df)

    summary = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in quality_df.columns])),
        html.Tbody([
            html.Tr([html.Td(val) for val in row])
            for row in quality_df.values
        ])
    ], style={'width': '100%', 'border': '1px solid black', 'borderCollapse': 'collapse'})

    fig_missing = px.bar(
        quality_df,
        x='Column',
        y='Missing (%)',
        title='Missing Values (%) by Column',
        text='Missing (%)',
        color='Missing (%)',
        color_continuous_scale='reds'
    )

    fig_unique = px.bar(
        quality_df,
        x='Column',
        y='Unique Values',
        title='Unique Values Count by Column',
        text='Unique Values',
        color='Unique Values',
        color_continuous_scale='blues'
    )

    return summary, fig_missing, fig_unique

# Run server
if __name__ == '__main__':
    app.run(debug=True)